In [0]:
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np


#### Problem 6 a b c d e and f 
### Reading the image

In [0]:

A=cv2.imread('apple.jpeg',1)
A=cv2.cvtColor(A,cv2.COLOR_BGR2RGB)
B=cv2.imread('orange.jpeg',1)
B=cv2.cvtColor(B,cv2.COLOR_BGR2RGB)

A=A.astype('float64')
B=B.astype('float64')

##### Performing the blur computations


In [0]:

G = A.copy()
gpA = [G]
lpA = []

### Function that computes the laplacian and guassian pyramid


In [0]:
def pyramidDown(G):
    gpA=[G]
    lpA=[]
    for i in range (6):
        
        G=cv2.GaussianBlur(gpA[i],(5,5),0)
        Gdiff=gpA[i]-G
        Gdown=G
        col=int(np.ceil(Gdown.shape[1]/2))
        row=int(np.ceil(Gdown.shape[0]/2))
        Gdown=cv2.resize(G,(col,row))
        
        Goutput = G
        Goutput = cv2.normalize(Goutput, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_64F)
        Gdownop = Gdown
        Gdownop = cv2.normalize(Gdownop, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_64F)
        
        plt.imshow(Gdownop)
        plt.title('Gaussian Pyramid Image')
        plt.show()
        Gdiffop = Gdiff
        Gdiffop = cv2.normalize(Gdiffop, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_64F)
        plt.imshow(Gdiffop,'gray')
        plt.title('Laplacian Pyramid Image')
        plt.show()
        gpA.append(Gdown)
        lpA.append(Gdiff)

    return gpA,lpA

### Function that does the multiresolution blend when it gets the guassians and laplacians as arguments

In [0]:
def multiresolutionblend(guA,laA,guB,laB,guM,laM):
    
    LaSoutput=[]
    GaSoutput=[]

    for i in range(0,len(laA),1):
        LaSoutput.append((1-guM[i])*laA[i] + guM[i]*laB[i])
        print(len(LaSoutput))
        for i in range(6,-1,-1):
                if i==6:
                    firstoutput = (1-guM[i])*guA[i] + guM[i]*guB[i]
                    GaSoutput.append(firstoutput)
                    continue
        print(len(GaSoutput))
        GaSoutput.append(cv2.resize(GaSoutput[len(GaSoutput)-1],(LaSoutput[i].shape[1],LaSoutput[i].shape[0])) + LaSoutput[i])
        
    return LaSoutput,GaSoutput


In [0]:
guA , laA = pyramidDown(A)

guB , laB = pyramidDown(B)

### Creating the mask variable to be used in the multiresolution blend

In [0]:
mask1=np.zeros((300,150,3))
mask2=np.ones((300,150,3))
mask3 = np.hstack((mask1,mask2))

guM , laM = pyramidDown(mask3)
print(' Length check ')
print(len(guM),len(laA))

Lblendop,Gblendop = multiresolutionblend(guA,laA,guB,laB,guM,laM)
print(len(Gblendop))


### The following loop prints the orapple at various stages in the pyramid
## The final image is the orapple produced

In [0]:
for i in range(len(Gblendop)):
    plt.imshow(np.uint8(Gblendop[i]))
    plt.show()

### Direct blending 6 d
## This combines the images together directly

In [0]:
directblend= (1- mask3)*A + mask3*B

print (directblend.dtype)
plt.imshow(np.uint8(directblend))
plt.title('Direct Blend')
plt.show()

### Alpha Blending 6 e
### This does a guassian blur on the mask first
### Then combines the images together

In [0]:
alphamask = cv2.GaussianBlur(mask3,(5,5),0)

alphablend = (1-alphamask)*A + alphamask*B
print (alphablend.dtype)
plt.imshow(np.uint8(alphablend))
plt.title('Aplha Blend')
plt.show()